In [1]:
from sepal_ui import widgetFactory as wf

#create the asset variable object 
gfc_aoi_IO = wf.Aoi_IO()

#create the tile 
gfc_aoi = wf.TileAOI(gfc_aoi_IO)

#this tile will only be displayed if voila is launch from this notebook 
gfc_aoi

Layout(children=[Card(children=[Html(children=['AOI selection'], tag='h2'), Layout(children=[Flex(children=[La…